# Environment
## 1.1 Creating Python3.5 Environment in Anaconda

In [1]:
pwd

'/Users/ruijisun/Documents/GitHub/mortar-analytics/zone_comfort_evaluation'

In [2]:
!python --version

Python 3.8.5


## 1.2 Installing Mortar 1.0.5 using pip (pip should be updated first)


In [4]:
!pip list

Package                            Version             Location
---------------------------------- ------------------- ----------------------------------------------------
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 2.0.4
anaconda-project                   0.8.3
applaunchservices                  0.2.1
appnope                            0.1.0
appscript                          1.1.1
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.4.2
astropy                            4.0.2
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.4
Babel                              2.8.1
backcall                           0.2.0
backports.functools-lru-cache      1.6.1
backports.shutil-get-terminal-size 1.0.0
backp

In [3]:
!pip --version

pip 20.2.4 from /opt/anaconda3/lib/python3.8/site-packages/pip (python 3.8)


Installing 1.0.5 pymortar pacakge

In [ ]:
!pip install pymortar>=2.0.0

In [15]:
!pip list

DEPRECATION: Python 3.5 reached the end of its life on September 13th, 2020. Please upgrade your Python as Python 3.5 is no longer maintained. pip 21.0 will drop support for Python 3.5 in January 2021. pip 21.0 will remove support for this functionality.
Package                  Version
------------------------ ---------
appnope                  0.1.0
bleach                   3.3.0
certifi                  2020.6.20
decorator                5.0.9
defusedxml               0.7.1
entrypoints              0.2.3
googleapis-common-protos 1.52.0
grpcio                   1.38.1
grpcio-tools             1.38.1
ipykernel                4.10.0
ipython                  5.8.0
ipython-genutils         0.2.0
Jinja2                   2.11.3
json5                    0.9.6
jsonschema               2.6.0
jupyter-client           5.3.3
jupyter-core             4.5.0
jupyterlab               1.2.6
jupyterlab-server        1.0.0
MarkupSafe               1.0
mistune                  0.8.3
nbconvert          

# 2 Application
## 2.1 Running the application developed by Carlos

In [8]:
import app

In [16]:
app.query_and_qualify()

INFO:root:Generating a new JWT token. Your old token may have expired
INFO:root:Got event ChannelConnectivity.IDLE
INFO:root:Got event ChannelConnectivity.CONNECTING
INFO:root:Got event ChannelConnectivity.READY


_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Received http2 header with status: 504"
	debug_error_string = "{"created":"@1625676081.538943000","description":"Received http2 :status header with non-200 OK status","file":"src/core/ext/filters/http/client/http_client_filter.cc","file_line":133,"grpc_message":"Received http2 header with status: 504","grpc_status":14,"value":"504"}"
>

## 2.2 Running functions step by step

In [19]:
import pymortar
import os
import pandas as pd

log account into the Mortar data server

In [21]:
import logging
logging.getLogger().setLevel("CRITICAL")
logging.getLogger('matplotlib').setLevel("CRITICAL")

client = pymortar.Client({
    'mortar_address': 'api.mortardata.org', # do not change
    'username': "ruijislife",  # <------------- CHANGE THIS
    'password': "Mortardataset!9957",  # <------------- CHANGE THIS
})
print("Success! Got client")

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Received http2 header with status: 504"
	debug_error_string = "{"created":"@1625676499.562551000","description":"Received http2 :status header with non-200 OK status","file":"src/core/ext/filters/http/client/http_client_filter.cc","file_line":133,"grpc_message":"Received http2 header with status: 504","grpc_status":14,"value":"504"}"
>

Build query to return zone air temperature measurements and qualify which sites can run this application

In [2]:
query = dict()

# define sensor type and query
sensor = "Zone_Air_Temperature"

sensor_query = """SELECT ?sensor WHERE {{
    ?sensor    rdf:type/rdfs:subClassOf*     brick:{0}_Sensor .
}};""".format(sensor)

#???? why using .format (sensor)?

In [ ]:
# find sites that have the zone air temperature
qualify_resp = client.qualify([sensor_query])
if qualify_resp.error != "":
    print("ERROR: ", qualify_resp.error)
    os.exit(1)

# save query and sensor used
query['query'] = sensor_query
query['sensor'] = sensor

print("Running on {0} sites".format(len(qualify_resp.sites)))

my own code for finding qualified sites

In [ ]:
qualified_sites = client.qualify([sensor_query])
n_qualified_sites = len(qualified_sites)
print(f"There are {n_qualified_sites} qualified sites in Mortar dataset.")

This query fetches the building-level electric meters for all available sites in the Mortar testbed, downsampled to a 15-min interval. The query meter_query does not pull in any additional context, and the View and DataFrame definitions are minimal.

In [ ]:
meter_query = """SELECT ?meter WHERE {
    ?meter rdf:type/rdfs:subClassOf* brick:Building_Electric_Meter
};"""

meter_view = pymortar.View(
    name="meters",
    definition=meter_query,
)

meter_data = pymortar.DataFrame(
    name="meters",
    aggregation=pymortar.MEAN,
    window="15m",
    timeseries=[
        pymortar.Timeseries(
            view="meters",
            dataVars=["?meter"],
        )
    ]
)

time_range = pymortar.TimeParams(
    start="2018-01-01T00:00:00Z",
    end="2019-01-01T00:00:00Z",
)

In [ ]:
qualified_sites = client.qualify([meter_query])

# define the request
request = pymortar.FetchRequest(
    sites = qualified_sites.sites,
    views = [meter_view],
    dataFrames = [meter_data],
    time = time_range,
)

# download the data
meter_resp = client.fetch(request)
print(f'Downloaded data: {meter_resp}')

### Downloaded Metadata

In [ ]:
meter_resp.view('meters').head()

### Downloaded Data

In [ ]:
meter_resp['meters'].head()

# the functions of the app:
1. query and qualify
2. fetch data
3. calculate thermal comfort (write below)

In [ ]:

# pymortar 2.0.4
class Client:
    """Method to create a new Pymortar client

    The configuration takes the following (optional) parameters:

    - mortar_address: address + port to connect to, e.g. "localhost:9001". Defaults to $MORTAR_API_ADDRESS from the environment. Currently expects a TLS-secured endpoint
    - username: your Mortar API username. Defaults to MORTAR_API_USERNAME env var
    - password: your Mortar API password. Defaults to MORTAR_API_PASSWORD env var

    Keyword Args:
        cfg (dict or None): configuration dictionary. Takes the following (optional) keys:

    Returns:
        client (Client): PyMortar client
    """
    def __init__(self, cfg=None):
        if cfg is not None:
            self._cfg = cfg
        else:
            self._cfg = {}

        # get username/password from environment or config file
        if 'username' not in self._cfg or not self._cfg['username']:
            self._cfg['username'] = os.environ.get('MORTAR_API_USERNAME')
        if 'password' not in self._cfg or not self._cfg['password']:
            self._cfg['password'] = os.environ.get('MORTAR_API_PASSWORD')

        if self._cfg.get('mortar_address') is None:
            self._mortar_address = os.environ.get('MORTAR_API_ADDRESS','mortardata.org:9001')
        else:
            self._mortar_address = self._cfg.get('mortar_address')

        self._connect()

        def connectivity_event_callback(event):
            if event == grpc.ChannelConnectivity.TRANSIENT_FAILURE:
                logging.error("Transient failure detected; reconnecting in 10 sec")
                time.sleep(10)
                self._connect()
            else:
                logging.info("Got event {0}".format(event))

        # listen to channel events
        self._channel.subscribe(connectivity_event_callback)

        if os.path.exists(".pymortartoken.json"):
            self._token = json.load(open(".pymortartoken.json", "r"))
            logging.info("loaded .pymortartoken.json token")
        else:
            self._token = None

        # TODO: handle the refresh token recycling automatically for the user
        # TODO: break this out into a method that can be called when we notice that the token is expired

        # FIX: comment this out to allow token to expire and see connection state
        if self._token is None:
            self._refresh()